# 3D analysis open problem

1) Proceed with the basic imports from the 3D analysis notebook:

In [1]:
import matplotlib.pyplot as plt

from pathlib import Path
import numpy as np
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion

from gammapy.catalog import SourceCatalogGammaCat
from gammapy.datasets import MapDataset, Datasets
from gammapy.data import (
    DataStore,
    Observation,
    Observations,
)
from gammapy.estimators import ExcessMapEstimator, TSMapEstimator
from gammapy.estimators.utils import find_peaks, find_peaks_in_flux_map
from gammapy.makers import (
    MapDatasetMaker, 
    SafeMaskMaker, 
    FoVBackgroundMaker
)
from gammapy.maps import MapAxis, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    ExpCutoffPowerLawSpectralModel,
    FoVBackgroundModel,
    GaussianSpatialModel,
    PointSpatialModel,
    PowerLawSpectralModel,
    SkyModel,
    Model,
    Models,
)

Silence some warnings that may affect your outputs:

In [2]:
# to silence the warnings
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)

2) Import the index files with the `DataStore` class from the directory with the `from_dir` function. 

In [5]:
data_store = 

SyntaxError: invalid syntax (1257863425.py, line 1)

3) Search around the Galactic coordinates l,b = 341.90669676°, -3.62457011°, and create the astropy `SkyCoord` object:

In [5]:
#source coordinates
pointing = 

4) Create the dictionary for the observation selection as in the 3D analysis notebook, and pick them with the `select_observation` function of `DataStore`:

In [6]:
selection = dict(
    
)
selected_obs_table = data_store.

5) Take them with the `get_observations` function:

In [2]:
observations = data_store.
len(observations)

6) Define the reconstructed and true energy axes (with `MapAxis`) and the geometry (with `WcsGeom.create`), as in the 3D analysis notebook:

In [9]:
# Reduced IRFs are defined in true energy (i.e. not measured energy).
energy_axis_true = 

energy_axis = 

geom = 

7) Create the MapDataset:

In [ ]:
stacked = 

8) Create the exclusion masks as in the 3D analysis notebook:

In [10]:
exclusion_src1 = 

exclusion_src2 = 

exclusion_src3 = 

exclusion_src4 = 

exclusion_src5 = 

exclusion_src6 = 

exclusion_geom = WcsGeom....
)

exclusion_mask = 

exclusion_mask.plot()
plt.show()

9) Set the safe ranges with `MapDatasetMaker`, `SafeMaksMaker` and `FoVBackgroundMaker`:

In [12]:
offset_max =
maker = 
maker_safe_mask = 


maker_fov = 

10) Loop over the observations to stack the datasets:

In [3]:
for i, obs in enumerate(observations):
    # First a cutout of the target map is produced
    cutout = 
    
    # A MapDataset is filled in this cutout geometry
    dataset = 
    
    # The data quality cut is applied
    dataset = 
    
    # fit background model
    dataset = 
    print(
        f"{i} of {len(observations)}, Background norm obs {obs.obs_id}: {dataset.background_model.spectral_model.norm.value:.2f}"
    )
    # The resulting dataset cutout is stacked onto the final one
    stacked.stack...

print(stacked)

11) Check if the sky-map is correct by plotting it:

In [4]:
stacked.peek()

12) Define a spatial and spectral model (with `PointSpatialModel`, `PowerLawSpectralModel` and `SkyModel`) to run the source detection: 

In [15]:
spatial_model = 
spectral_model =

model = 

12) Use the `TSMapEstimator` class to perform the source detection:

In [16]:
estimator = 

maps = 

13) Plot the maps of the "sqrt_ts" and "flux":

In [15]:
fig, (ax1, ax2, ax3) = plt.subplots(
    ncols=3,
    figsize=(20, 3),
    subplot_kw={"projection": stacked.counts.geom.wcs},
    gridspec_kw={"left": 0.1, "right": 0.98},
)

maps["sqrt_ts"].plot(ax=ax1, add_cbar=True)
ax1.set_title("Significance map")
maps["flux"].plot(ax=ax2, add_cbar=True, stretch="sqrt", vmin=0)
ax2.set_title("Flux map")
maps["niter"].plot(ax=ax3, add_cbar=True)
ax3.set_title("Iteration map")
plt.show()

13) Find the peaks of "sqrt_ts" with the function `find_peaks` in `gammapy.estimators.utils`. Sources can be significant with a 5 sigma detection:

In [5]:
sources = find_peaks...

In [ ]:
14) Make a catalog and then compare it with any catalog you can find in Gammapy:

15) Plot the maps overplotting the source regions.

16) Set the background model with `FoVBackgroundModel`. Set its normalization to 1:

In [19]:
bkg_model = 

bkg_model.spectral_model.norm.value = 1.0
    
models_stacked = Models...

17) Loop over each source and perform a 3D analysis. Use a simple `PowerLawSpectralModel` and a generic `GaussianSpatialModel` for the spectral and morphological properties (for simplicity, freeze the coordinate parameters):

In [7]:
for i, src in enumerate(sources):
    print(f"- Analysis of the source {i+1}...")

    spectral_model = 
    
    spatial_model = 

    model = 

    models_stacked.extend(Models([model]))
    
    stacked.models 

    stacked.models[-1].parameters["lon_0"].frozen = True
    stacked.models[-1].parameters["lat_0"].frozen = True

    fit = 
    result = 
    print(result)
    print(f"   * Analysis of the source {i+1}... complete!")

18) Use the class `ExcessMapEstimator` to investigate the excess. If the modelization is suitable, the maps will be flat:

In [16]:
estimator = ExcessMapEstimator


result = 

result["sqrt_ts"].plot_grid(
    figsize=(12, 12), cmap="coolwarm", add_cbar=True, vmin=-5, vmax=5, ncols=2
)
plt.show()